In [2]:
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
df = pd.read_csv('./data/bestSelling_games.csv')
df.head()

,game_name,reviews_like_rate,all_reviews_number,release_date,developer,user_defined_tags,supported_os,supported_languages,price,other_features,age_restriction,rating,difficulty,length,estimated_downloads
0,Counter-Strike 2,86,8803754,"21 Aug, 2012",Valve,"FPS, Action, Tactical","win, linux","English, Czech, Danish, Dutch, Finnish, French...",0.00,"Cross-Platform Multiplayer, Steam Trading Card...",17,3.20,4,80,306170000
1,PUBG: BATTLEGROUNDS,59,2554482,"21 Dec, 2017",PUBG Corporation,"Survival, Shooter, Action, Tactical",win,"English, Korean, Simplified Chinese, French, G...",0.00,"Online PvP, Stats, Remote Play on Phone, Remot...",13,3.10,4,73,162350000
2,ELDEN RING NIGHTREIGN,77,53426,"30 May, 2025","FromSoftware, Inc.","Souls-like, Open World, Fantasy, RPG",win,"English, Japanese, French, Italian, German, Sp...",25.99,"Single-player, Online Co-op, Steam Achievement...",17,3.96,4,50,840000
3,The Last of Us™ Part I,79,45424,"28 Mar, 2023",Naughty Dog LLC,"Story Rich, Shooter, Survival, Horror",win,"English, Italian, Spanish - Spain, Czech, Dutc...",59.99,"Single-player, Steam Achievements, Steam Tradi...",17,4.10,3,24,2000000
4,Red Dead Redemption 2,92,672140,"5 Dec, 2019",Rockstar Games,"Open World, Story Rich, Adventure, Realistic, ...",win,"English, French, Italian, German, Spanish - Sp...",59.99,"Single-player, Online PvP, Online Co-op, Steam...",17,4.32,3,80,21610000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   game_name            2380 non-null   object 
 1   reviews_like_rate    2380 non-null   int64  
 2   all_reviews_number   2380 non-null   int64  
 3   release_date         2380 non-null   object 
 4   developer            2380 non-null   object 
 5   user_defined_tags    2380 non-null   object 
 6   supported_os         2380 non-null   object 
 7   supported_languages  2380 non-null   object 
 8   price                2380 non-null   float64
 9   other_features       2380 non-null   object 
 10  age_restriction      2380 non-null   int64  
 11  rating               2380 non-null   float64
 12  difficulty           2380 non-null   int64  
 13  length               2380 non-null   int64  
 14  estimated_downloads  2380 non-null   int64  
dtypes: float64(2), int64(6), object(7)
mem

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
df.describe(include='all')


,game_name,reviews_like_rate,all_reviews_number,release_date,developer,user_defined_tags,supported_os,supported_languages,price,other_features,age_restriction,rating,difficulty,length,estimated_downloads
count,2380,"2,380.00","2,380.00",2380,2380,2380,2380,2380,"2,380.00",2380,"2,380.00","2,380.00","2,380.00","2,380.00","2,380.00"
unique,2380,NaN,NaN,1511,1806,1649,6,1606,NaN,761,NaN,NaN,NaN,NaN,NaN
top,UBERMOSH:OMEGA,NaN,NaN,"22 May, 2025",Valve,"Simulation, Management",win,English,NaN,"Single-player, Steam Achievements, Steam Cloud...",NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,15,17,29,1724,323,NaN,275,NaN,NaN,NaN,NaN,NaN
mean,NaN,82.41,"31,615.08",NaN,NaN,NaN,NaN,NaN,10.51,NaN,10.61,3.23,2.86,22.97,"2,523,006.58"
std,NaN,12.64,"213,719.68",NaN,NaN,NaN,NaN,NaN,11.34,NaN,6.40,0.77,0.98,21.25,"11,182,829.61"
min,NaN,20.00,10.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,0.39,1.00,1.00,90.00
25%,NaN,76.00,342.00,NaN,NaN,NaN,NaN,NaN,2.99,NaN,10.00,2.75,2.00,6.00,"35,000.00"
50%,NaN,85.00,"2,106.50",NaN,NaN,NaN,NaN,NaN,7.99,NaN,13.00,3.38,3.00,16.00,"217,150.00"
75%,NaN,92.00,"13,030.00",NaN,NaN,NaN,NaN,NaN,12.49,NaN,17.00,3.81,3.00,34.00,"1,380,000.00"


## Dataset Overview

This dataset is retrived from kaggle: [Best-Selling Steam Games of All Time](https://www.kaggle.com/datasets/hbugrae/best-selling-steam-games-of-all-time)

This dataset contains information about **2,380 best-selling games on Steam**, it offers comprehensive look into various aspects of each game. It includes both quantitative and qualitative features, which gives a good foundation for analysis.

---

#### Key numerical features include:

+ `reviews_like_rate`: The percentage of positive reviews, ranging from 20% to a perfect 100%, with an average of approximately 82.41%.
    * According to Kaggle Dataset documentation:
    > reviews_like_rate: The recommendation rate from user reviews on Steam (e.g., '95% of the 100 reviews are positive').

* `all_reviews_number`: The total number of reviews, which varies widely from 10 to over 8.8 million, emphasizing a significant variation in player engagement.

+ `price`: Game prices range from free (0.00) up to 79.99, with an average price of around 10.51, suggest that there could be a  diverse pricing strategy among best-selling titles.
    * According to Kaggle Dataset documentation:
    > price: The price of the game. Note: The currency is MENA - U.S. Dollar, a regional price for the Middle East & North Africa, not the standard USD. A value of 0 in this column indicates the game is 'Free to Play'.

* `estimated_downloads`: Ranging from a mere 90 to an whopping 306 million, this feature highlights the massive difference in market penetration among these games.

* `age_restriction`: With values from 0 to 17, this indicates the recommended age for players.
    * According to Kaggle Dataset documentation:
    > age_restriction: The recommended age restriction for the game's content, encoded as follows: 0 (Everyone), 10 (10+), 13 (13+), 17 (17+).

+ `difficulty`: A numerical rating from 1 to 5, providing insight into the perceived challenge of the games, averaging around 2.86.
    * According to Kaggle Dataset documentation:
    > difficulty: An estimated difficulty of the game as perceived by players, on a scale of 1 to 5, where 1 is the easiest and 5 is the hardest.

+ `length`: Represents the estimated playtime in hours, varying from 1 to 80 hours.
    * According to the Kaggle Dataset documentation:
    > length: The average time (in hours) players spend to complete or fully experience the game. For this dataset, the value is capped at a maximum of 80 hours.

+ `rating`: An average rating score ranging from 0.39 to 4.83.
    * According to the Kaggle Dataset documentation:
    > rating: An overall user-provided rating for the game on a scale of 1 to 5, where 1 is the lowest and 5 is the highest.

---

#### Categorical and textual features provide additional context:

* `game_name`: Unique identifiers for each game.

* `release_date`: The date the game was released, which will be crucial for calculating *game age*.

* `developer`: The studio responsible for the game, with 1806 unique developers in the dataset.

* `user_defined_tags`: A critical field containing multiple descriptive tags (e.g., 'FPS', 'Action', 'Tactical'), *which will require parsing for genre analysis*.

* `supported_os`: Indicates the operating systems supported (e.g., 'win', 'linux'), with 'win' being the most common.

* `supported_languages`: The languages supported by the game, with English being the most frequent.

* `other_features`: Lists additional functionalities like multiplayer options or Steam achievements.

The dataset contains **no missing values**, which simplifies the initial cleaning process.

## 1. Data Cleaning & Pre-processing
To maintain the quality of the data, following measures will be considerd

1. Handle any inconsistencies and format the data correctly for analysis.

2. Handle Missing Values: check columns like `difficulty`, `length`, etc., for any NaN values and decide on an appropriate strategy, such as imputation (e.g., filling with the mode) or removal.

3. Data Type Conversion: The `release_date` column is currently an object. It will be converted to a datetime object to enable time-based analysis, such as calculating the game's age.

4. Column Cleanup: standardize column names (e.g., removing spaces or special characters) if necessary for easier access.

In [10]:
df.isnull().sum()

game_name              0
reviews_like_rate      0
all_reviews_number     0
release_date           0
developer              0
user_defined_tags      0
supported_os           0
supported_languages    0
price                  0
other_features         0
age_restriction        0
rating                 0
difficulty             0
length                 0
estimated_downloads    0
dtype: int64

No Null or NaN values found on each column. Therefore no need of applying imputation

In [11]:
df['release_date'] = pd.to_datetime(df['release_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   game_name            2380 non-null   object        
 1   reviews_like_rate    2380 non-null   int64         
 2   all_reviews_number   2380 non-null   int64         
 3   release_date         2380 non-null   datetime64[ns]
 4   developer            2380 non-null   object        
 5   user_defined_tags    2380 non-null   object        
 6   supported_os         2380 non-null   object        
 7   supported_languages  2380 non-null   object        
 8   price                2380 non-null   float64       
 9   other_features       2380 non-null   object        
 10  age_restriction      2380 non-null   int64         
 11  rating               2380 non-null   float64       
 12  difficulty           2380 non-null   int64         
 13  length               2380 non-nul

## 2. Feature Engineering
To get the most out of our dataset, we will create new, more informative features from the existing ones.

Genre & Tag Processing: The user_defined_tags column is a text field containing multiple tags. We will parse this field to extract the most frequent and relevant tags (e.g., 'Indie', 'Action', 'RPG') and convert them into binary features (One-Hot Encoding).

Create 'Game Age': Using the converted release_date column, we will calculate the age of each game in years. This can be a powerful feature for segmentation.

Create 'Review Ratio': We can create a more robust popularity metric by combining the like rate and the total number of reviews (e.g., reviews_like_rate * all_reviews_number).

In [12]:
# Create 'Game Age': Calculate the age of each game in years
current_year = pd.to_datetime('today').year
df['game_age'] = current_year - df['release_date'].dt.year
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   game_name            2380 non-null   object        
 1   reviews_like_rate    2380 non-null   int64         
 2   all_reviews_number   2380 non-null   int64         
 3   release_date         2380 non-null   datetime64[ns]
 4   developer            2380 non-null   object        
 5   user_defined_tags    2380 non-null   object        
 6   supported_os         2380 non-null   object        
 7   supported_languages  2380 non-null   object        
 8   price                2380 non-null   float64       
 9   other_features       2380 non-null   object        
 10  age_restriction      2380 non-null   int64         
 11  rating               2380 non-null   float64       
 12  difficulty           2380 non-null   int64         
 13  length               2380 non-nul

In [25]:
# Genre & Tag Processing: Extract and One-Hot Encode the most frequent tags
# First, get all unique tags
all_tags = df['user_defined_tags'].str.split(', ').explode()
# Get the most frequent tags (e.g., top 20, you can adjust this number)
top_tags = all_tags.value_counts().head(1649).index.tolist()
df.describe(include='all')
## print top_tags with count
print("Top Tags and their counts:")
print(all_tags.value_counts())

Top Tags and their counts:
user_defined_tags
Simulation        736
Action            730
Adventure         522
RPG               437
Strategy          353
2D                308
Horror            299
FPS               246
Survival          237
Open World        236
Puzzle            221
Management        201
Early Access      199
Roguelike         177
Building          176
Story Rich        169
Sandbox           164
Platformer        156
Combat            145
Shooter           144
Turn-Based        131
Anime             129
Cute              128
Driving           112
Fantasy           100
Pixel Graphics     94
Visual Novel       93
Tactical           92
War                89
Sexual Content     89
Sports             83
Crafting           78
Stealth            67
Realistic          60
Fighting           59
Card Game          48
JRPG               44
Souls-like         40
Automation         36
MMORPG             31
2.5D               16
CRPG               11
FPS ,RPG            1
Adventure

In [27]:
all_tags = df['user_defined_tags'].str.split(', ').explode()
unique_tags = all_tags.unique()

print("All unique tags:")
for tag in unique_tags:
    print(tag)

# Or, to print as a list:
# print("\nAll unique tags (as a list):")
# print(unique_tags.tolist())

All unique tags:
FPS
Action
Tactical
Survival
Shooter
Souls-like
Open World
Fantasy
RPG
Story Rich
Horror
Adventure
Realistic
Sandbox
Strategy
Combat
Sports
Simulation
Early Access
2D
Crafting
Driving
Stealth
Building
Management
Visual Novel
War
Turn-Based
JRPG
MMORPG
Anime
Roguelike
Automation
Cute
Platformer
Card Game
Fighting
Pixel Graphics
Sexual Content
FPS ,RPG
Adventure ,RPG
2.5D
Puzzle
CRPG
Simulation,

Tactical,
RPG,


In [28]:
print("\nAll unique tags (as a list - might be truncated by default Python print):")
print(unique_tags.tolist())



All unique tags (as a list - might be truncated by default Python print):
['FPS', 'Action', 'Tactical', 'Survival', 'Shooter', 'Souls-like', 'Open World', 'Fantasy', 'RPG', 'Story Rich', 'Horror', 'Adventure', 'Realistic', 'Sandbox', 'Strategy', 'Combat', 'Sports', 'Simulation', 'Early Access', '2D', 'Crafting', 'Driving', 'Stealth', 'Building', 'Management', 'Visual Novel', 'War', 'Turn-Based', 'JRPG', 'MMORPG', 'Anime', 'Roguelike', 'Automation', 'Cute', 'Platformer', 'Card Game', 'Fighting', 'Pixel Graphics', 'Sexual Content', 'FPS ,RPG', 'Adventure ,RPG', '2.5D', 'Puzzle', 'CRPG', 'Simulation,', '', 'Tactical,', 'RPG,']


In [29]:
# Create binary features for each top tag
for tag in top_tags:
    df[f'tag_{tag.replace(" ", "-").replace("-", "-")}'] = df['user_defined_tags'].apply(lambda x: 1 if tag in x else 0)

# Create 'Review Ratio': reviews_like_rate * all_reviews_number
df['review_ratio'] = df['reviews_like_rate'] * df['all_reviews_number']

print(top_tags)


['Simulation', 'Action', 'Adventure', 'RPG', 'Strategy', '2D', 'Horror', 'FPS', 'Survival', 'Open World', 'Puzzle', 'Management', 'Early Access', 'Roguelike', 'Building', 'Story Rich', 'Sandbox', 'Platformer', 'Combat', 'Shooter', 'Turn-Based', 'Anime', 'Cute', 'Driving', 'Fantasy', 'Pixel Graphics', 'Visual Novel', 'Tactical', 'War', 'Sexual Content', 'Sports', 'Crafting', 'Stealth', 'Realistic', 'Fighting', 'Card Game', 'JRPG', 'Souls-like', 'Automation', 'MMORPG', '2.5D', 'CRPG', 'FPS ,RPG', 'Adventure ,RPG', 'Simulation,', '', 'Tactical,', 'RPG,']


## 3. Exploratory Data Analysis (EDA)
With clean data, we will now explore relationships and patterns through visualization to understand the dataset's structure.

Distribution Analysis: We will create histograms and box plots for key numeric features like price, estimated_downloads, and the newly created game_age to understand their distributions and identify outliers.

Relationship Analysis: Scatter plots will be used to visualize the relationships between pairs of variables, such as price vs. reviews_like_rate, to see if natural clusters appear visually.

Genre Popularity: A bar chart will be created from the new tag features to visualize the most common game genres in the best-seller list.

## 4. Hopkins Statistic & Data Scaling
Before applying a clustering algorithm, we must check if the data has a natural tendency to be clustered and then scale it.

Assess Clustering Tendency: We will calculate the Hopkins statistic. A value close to 1 indicates that the data is highly clusterable, justifying our use of clustering algorithms.

Feature Scaling: Since clustering algorithms like K-Means are distance-based, features must be on a similar scale. We will use StandardScaler from scikit-learn to scale our selected numeric features to have a mean of 0 and a standard deviation of 1. This ensures that no single feature dominates the clustering process.